# Docker Containers with SSH

### SSH Client

In [1]:
import paramiko
from scp import SCPClient

def create_ssh_client(hostname, port, username, password):
    client = paramiko.SSHClient()
    client.load_system_host_keys()
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    client.connect(hostname, port=port, username=username, password=password)
    return client

def send_files_scp(client, local_path, remote_path, source_files, target_files):
    if len(source_files) != len(target_files):
        print("The number of source files is not equal number of targets.")
        return
    with SCPClient(client.get_transport()) as scp:
        for src, dest in zip(source_files, target_files):
            local_file_path = local_path + "/" + src
            remote_file_path = remote_path + "/" + dest
            print(f"Sending {local_file_path} to {remote_file_path}")
            scp.put(local_file_path, remote_file_path)  

### Consts

In [2]:
LOCAL_DIRECTORY = '.'
REMOTE_DIRECTORY = '/opt/contiki-ng/tools/cooja'
CONTAINER_HOSTNAME = 'localhost'
CONTAINER_USERNAME = 'root'
CONTAINER_PASSWORD = 'root'  

LOCAL_FILES = [
    "temp/simulation.xml", 
    "temp/positions.dat",
    "data/Makefile", 
    "data/project-conf.h",
    "data/udp-client.c", 
    "data/udp-server.c"]

REMOTE_FILES = [
    "simulation.csc", 
    "positions.dat",
    "Makefile", 
    "project-conf.h",
    "udp-client.c", 
    "udp-server.c"]

### Send basic data

In [4]:
for port in range(2231, 2236):
    print(f"Sending for port {port}")
    ssh = create_ssh_client(CONTAINER_HOSTNAME, port, CONTAINER_USERNAME, CONTAINER_PASSWORD)
    send_files_scp(ssh, LOCAL_DIRECTORY, REMOTE_DIRECTORY, LOCAL_FILES, REMOTE_FILES)
    ssh.close()
    print("Transfer completed.")

Sending for port 2231
Sending ./temp/simulation.xml to /opt/contiki-ng/tools/cooja/simulation.csc
Sending ./temp/positions.dat to /opt/contiki-ng/tools/cooja/positions.dat
Sending ./data/Makefile to /opt/contiki-ng/tools/cooja/Makefile
Sending ./data/project-conf.h to /opt/contiki-ng/tools/cooja/project-conf.h
Sending ./data/udp-client.c to /opt/contiki-ng/tools/cooja/udp-client.c
Sending ./data/udp-server.c to /opt/contiki-ng/tools/cooja/udp-server.c
Transfer completed.
Sending for port 2232
Sending ./temp/simulation.xml to /opt/contiki-ng/tools/cooja/simulation.csc
Sending ./temp/positions.dat to /opt/contiki-ng/tools/cooja/positions.dat
Sending ./data/Makefile to /opt/contiki-ng/tools/cooja/Makefile
Sending ./data/project-conf.h to /opt/contiki-ng/tools/cooja/project-conf.h
Sending ./data/udp-client.c to /opt/contiki-ng/tools/cooja/udp-client.c
Sending ./data/udp-server.c to /opt/contiki-ng/tools/cooja/udp-server.c
Transfer completed.
Sending for port 2233
Sending ./temp/simulation.

### Running simulation and retrieving logs

In [ ]:
import scp
from pathlib import Path

# Caminhos remotos
SIMULATION_FILE = "simulation.csc"
REMOTE_LOG_FILE = "sim.log"
REMOTE_TESTLOG = "COOJA.testlog"
REMOTE_JAVA = "/opt/java/openjdk/bin/java"

# Caminhos locais
LOCAL_LOG_DIR = "logs"
Path(LOCAL_LOG_DIR).mkdir(exist_ok=True)

def run_cooja_simulation(port=2231):
    ssh = create_ssh_client(CONTAINER_HOSTNAME, port, CONTAINER_USERNAME, CONTAINER_PASSWORD)
    
    try:        
        # Comando a ser executado
        command = f"""
        cd ../{REMOTE_DIRECTORY} &&
        {REMOTE_JAVA} --enable-preview -Xms4g -Xmx4g -jar build/libs/cooja.jar --no-gui {SIMULATION_FILE}
        """
        
        # Executa o comando e captura a saída em tempo real
        stdin, stdout, stderr = ssh.exec_command(command)
        
        print("Simulação Cooja iniciada...")
        
        for line in iter(stdout.readline, ""):
            print(line, end="")  # Exibe a saída da simulação em tempo real
        
        exit_status = stdout.channel.recv_exit_status()
        print(f"Simulação concluída com status: {exit_status}")
        
        # Copia os arquivos via SCP
        with scp.SCPClient(ssh.get_transport()) as scp_client:
            scp_client.get(f"{REMOTE_DIRECTORY}/{REMOTE_TESTLOG}", f"{LOCAL_LOG_DIR}/{REMOTE_TESTLOG}")
        
        print(f"Arquivos copiados para {LOCAL_LOG_DIR}/")
        
    except Exception as e:
        print(f"Erro durante a execução: {str(e)}")
    finally:
        ssh.close()
        print("Conexão SSH encerrada")

if __name__ == "__main__":
    run_cooja_simulation()

Simulação Cooja iniciada...
INFO  [main] [Cooja.java:1145] - Loading simulation.csc random seed: null
> make -j8 udp-server.cooja TARGET=cooja
  LD        build/cooja/udp-server.cooja
> make -j8 udp-client.cooja TARGET=cooja
  LD        build/cooja/udp-client.cooja
INFO  [main] [LogScriptEngine.java:231] - Script timeout in 1501000 ms
INFO  [sim] [LogScriptEngine.java:320] -  5% completed, 125.5 sec remaining
INFO  [sim] [LogScriptEngine.java:320] - 10% completed, 104.4 sec remaining
INFO  [sim] [LogScriptEngine.java:320] - 15% completed, 89.1 sec remaining
INFO  [sim] [LogScriptEngine.java:320] - 20% completed, 80.6 sec remaining
INFO  [sim] [LogScriptEngine.java:320] - 25% completed, 79.5 sec remaining
INFO  [sim] [LogScriptEngine.java:320] - 30% completed, 77.0 sec remaining
INFO  [sim] [LogScriptEngine.java:320] - 35% completed, 151.9 sec remaining
INFO  [sim] [LogScriptEngine.java:320] - 40% completed, 131.1 sec remaining
INFO  [sim] [LogScriptEngine.java:320] - 45% completed, 120